In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
cd36_importances = pd.read_csv('/kaggle/input/research-project-01-around-multimodal-singlecell/CD36importances.csv')

In [ ]:
cd36_importances.head()

In [ ]:
cd36_importances_new = cd36_importances.loc[1:, :]
cd36_importances_new.columns = cd36_importances.loc[0, :]

In [ ]:
cd36_importances_new.head()

In [ ]:
train_cite_input_df = pd.read_hdf("/kaggle/input/open-problems-multimodal/train_cite_inputs.h5")
train_cite_target_df = pd.read_hdf("/kaggle/input/open-problems-multimodal/train_cite_targets.h5")

In [ ]:
train_cite_input_df.head()

In [ ]:
train_cite_target_df.head()

In [ ]:
dfc = train_cite_input_df.rename(columns=lambda x: x.split('_')[0])

In [ ]:
dfc.head()

In [ ]:
cd36_importances_new.head()

### SIM CD36 Correlation

In [ ]:
sim_cd_list = cd36_importances_new['SEMANTIC'].tolist()

In [ ]:
%%capture
!pip install mygene

In [ ]:
import mygene
mg = mygene.MyGeneInfo()

In [ ]:
sim_cd_list_out = mg.querymany(sim_cd_list, scopes='symbol', fields='ensembl.gene', species='human')

In [ ]:
df_sim_cd_list_out = pd.DataFrame(sim_cd_list_out, columns=['query', '_id', '_score', 'ensembl'])

In [ ]:
df_sim_cd_list_out['sym_ensembl'] = df_sim_cd_list_out.ensembl.astype(str).str.extract(r"{'gene': '(.*?)'}")

In [ ]:
df_sim_cd_list_out.head()

In [ ]:
sim_cd_ens_list = list(df_sim_cd_list_out['sym_ensembl'])

In [ ]:
dfc_sim = dfc[dfc.columns.intersection(sim_cd_ens_list)]

In [ ]:
dfc_sim.head()

In [ ]:
df_cd36 = train_cite_target_df[['CD36']]

In [ ]:
df_sim_corr = df_cd36.apply(dfc_sim.corrwith)

In [ ]:
df_sim_corr = df_sim_corr.reset_index()

In [ ]:
sim_cd_list = df_sim_corr['gene_id'].tolist()

In [ ]:
sim_cd_list_out = mg.querymany(sim_cd_list, scopes='ensembl.gene', fields='symbol', species='human')

In [ ]:
df_sim_cd_list_out = pd.DataFrame(sim_cd_list_out, columns=['query', '_id', '_score', 'symbol'])

In [ ]:
df_sim_cd_list_out.head()

In [ ]:
sim_df_merge = pd.merge(df_sim_corr, df_sim_cd_list_out, left_on='gene_id', right_on='query')

In [ ]:
sim_df_merge.head()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns

In [ ]:
sim_df_merge_filter = sim_df_merge[sim_df_merge['symbol'] != 'CD36']

In [ ]:
sim_df_merge_filter.to_csv('./sim_df_merge_filter.csv')

In [ ]:
figure(num=None, figsize=(10,20), dpi=80, facecolor='w', edgecolor='r')
sns.barplot(x="CD36", y = "symbol" , data = sim_df_merge_filter)
plt.xticks(rotation=90)
plt.title("Similarity 100 neighbors CD36 correlation")
plt.show()

### PPI CD36 Correlation

In [ ]:
ppi_cd_list = cd36_importances_new['PPI'].tolist()

In [ ]:
ppi_cd_list_out = mg.querymany(ppi_cd_list, scopes='symbol', fields='ensembl.gene', species='human')

In [ ]:
df_ppi_cd_list_out = pd.DataFrame(ppi_cd_list_out, columns=['query', '_id', '_score', 'ensembl'])

In [ ]:
df_ppi_cd_list_out['ppi_ensembl'] = df_ppi_cd_list_out.ensembl.astype(str).str.extract(r"{'gene': '(.*?)'}")

In [ ]:
ppi_cd_ens_list = list(df_ppi_cd_list_out['ppi_ensembl'])

In [ ]:
dfc_ppi = dfc[dfc.columns.intersection(ppi_cd_ens_list)]

In [ ]:
df_ppi_corr = df_cd36.apply(dfc_ppi.corrwith)

In [ ]:
df_ppi_corr = df_ppi_corr.reset_index()

In [ ]:
ppi_cd_list = df_ppi_corr['gene_id'].tolist()

In [ ]:
ppi_cd_list_out = mg.querymany(ppi_cd_list, scopes='ensembl.gene', fields='symbol', species='human')

In [ ]:
df_ppi_cd_list_out = pd.DataFrame(ppi_cd_list_out, columns=['query', '_id', '_score', 'symbol'])

In [ ]:
ppi_df_merge = pd.merge(df_ppi_corr, df_ppi_cd_list_out, left_on='gene_id', right_on='query')

In [ ]:
ppi_df_merge_filter = ppi_df_merge[ppi_df_merge['symbol'] != 'CD36']

In [ ]:
figure(num=None, figsize=(10,20), dpi=80, facecolor='w', edgecolor='r')
sns.barplot(x="CD36", y = "symbol" , data = ppi_df_merge_filter)
plt.xticks(rotation=90)
plt.title("PPI 100 neighbors CD36 correlation")
plt.show()

In [ ]:
ppi_df_merge_filter.to_csv('./ppi_df_merge_filter.csv')

### Сorrelation coefficients mean values

In [ ]:
abs(sim_df_merge_filter.CD36).mean()

In [ ]:
abs(ppi_df_merge_filter.CD36).mean()

### Сorrelation coefficients distribution

In [ ]:
import time

In [ ]:
plt.figure(figsize=(14,8))
plt.hist(sim_df_merge_filter.CD36, bins = 50)
plt.title('Sim correlation coefficients distribution')
plt.show()

In [ ]:
plt.figure(figsize=(14,8))
plt.hist(ppi_df_merge_filter.CD36, bins = 50)
plt.title('PPI correlation coefficients distribution')
plt.show()